In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_gjg60qr
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_gjg60qr
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-f5c6ndwy/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpad8bt_ry".


**Question1 (Q1)**

In [ ]:

%%writefile Q1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
using namespace std;

// Kernel definition
__global__ void ArrAdd(double *arr, int size,double *result)
{
    // declares a shared memory segment that is accessible by all threads in the same block. More on this later.
extern __shared__ double partialSum[];


unsigned int t = threadIdx.x;

unsigned int new_size = (size-1) / blockDim.x +1 ;
partialSum[t] = 0;
for(int i=0;i<new_size ;i++)
{

partialSum[t]+= arr[t*new_size+i];

}
//partialSum[t] = arr[t];

if(t >=size)
   return ;
for(unsigned int stride = 1; stride < blockDim.x; stride *= 2){

     if(t % (2*stride) == 0)
          partialSum[t] += partialSum[t+stride];
     __syncthreads();
}

 if(t==0)
     *result = partialSum[0];

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

 double* array = nullptr;  // Pointer to the array
 double * result = nullptr;
    double value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        double* temp = static_cast<double*>(realloc(array, (count + 1) * sizeof(double)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }



    // host to device allocation
     double *array_device;
     double *result_device;

     result = (double*)malloc(sizeof(double));
     cudaMalloc((void**)&array_device, count * sizeof(double));
     cudaMalloc((void**)&result_device,sizeof(double));

    cudaMemcpy(array_device, array, count * sizeof(double), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1, 1);
    ArrAdd<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(double) >>>(array_device,count,result_device);

    cudaMemcpy( result, result_device, sizeof(double), cudaMemcpyDeviceToHost);

    //cout<<"final sum is: "<<*result<<endl;
    cout << fixed <<setprecision(2) << *result <<endl;

    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Overwriting Q1_2_31_2_33.cu


In [ ]:

%%writefile Q1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
using namespace std;

// Kernel definition
__global__ void ArrAdd(double *arr, int size,double *result)
{
    // declares a shared memory segment that is accessible by all threads in the same block. More on this later.
extern __shared__ double partialSum[];


unsigned int t = threadIdx.x;

unsigned int new_size = (size-1) / blockDim.x +1 ;
partialSum[t] = 0;
for(int i=0;i<new_size ;i++)
{

partialSum[t]+= arr[t*new_size+i];

}
//partialSum[t] = arr[t];

if(t >=size)
   return ;
for(unsigned int stride = 1; stride < blockDim.x; stride *= 2){

     if(t % (2*stride) == 0)
          partialSum[t] += partialSum[t+stride];
     __syncthreads();
}

 if(t==0)
     *result = partialSum[0];

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

 double* array = nullptr;  // Pointer to the array
 double * result = nullptr;
    double value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        double* temp = static_cast<double*>(realloc(array, (count + 1) * sizeof(double)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }



    // host to device allocation
     double *array_device;
     double *result_device;

     result = (double*)malloc(sizeof(double));
     cudaMalloc((void**)&array_device, count * sizeof(double));
     cudaMalloc((void**)&result_device,sizeof(double));

    cudaMemcpy(array_device, array, count * sizeof(double), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1, 1);
    ArrAdd<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(double) >>>(array_device,count,result_device);

    cudaMemcpy( result, result_device, sizeof(double), cudaMemcpyDeviceToHost);

    //cout<<"final sum is: "<<*result<<endl;
    cout << fixed <<setprecision(2) << *result <<endl;

    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


# updated without prints


In [71]:

%%writefile Q1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
using namespace std;

// Kernel definition
__global__ void ArrAdd(double *arr, int size,double *result)
{
    // declares a shared memory segment that is accessible by all threads in the same block. More on this later.
extern __shared__ double partialSum[];


unsigned int t = threadIdx.x;

unsigned int new_size = (size-1) / blockDim.x +1 ;
partialSum[t] = 0;
for(int i=0;i<new_size ;i++)
{

partialSum[t]+= arr[t*new_size+i];

}
//partialSum[t] = arr[t];

if(t >=size)
   return ;
for(unsigned int stride = 1; stride < blockDim.x; stride *= 2){

     if(t % (2*stride) == 0)
          partialSum[t] += partialSum[t+stride];
     __syncthreads();
}

 if(t==0)
     *result = partialSum[0];

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

    double* array = nullptr;  // Pointer to the array
    double * result = nullptr;
    double value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        double* temp = static_cast<double*>(realloc(array, (count + 1) * sizeof(double)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // host to device allocation
     double *array_device;
     double *result_device;

     result = (double*)malloc(sizeof(double));
     cudaMalloc((void**)&array_device, count * sizeof(double));
     cudaMalloc((void**)&result_device,sizeof(double));

    cudaMemcpy(array_device, array, count * sizeof(double), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1, 1);
    ArrAdd<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(double) >>>(array_device,count,result_device);

    cudaMemcpy( result, result_device, sizeof(double), cudaMemcpyDeviceToHost);

    //cout<<*result<<endl;
    cout << fixed <<setprecision(2) << *result <<endl;

    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(result_device);

   return 0;
}


Writing Q1_2_31_2_33.cu


In [73]:
!chmod +x Q1_2_31_2_33.cu
!nvcc Q1_2_31_2_33.cu -o Q1

45.00


In [74]:
!./Q1 "input_10.txt"

45.00


In [75]:
!./Q1 "input_100.txt"

4950.00


In [76]:
!./Q1 "input_1000.txt"

499500.00


In [77]:
!./Q1 "input_10th.txt"

49995000.00


In [78]:
!./Q1 "input_100th.txt"

4999950000.00


In [79]:
!./Q1 "input_1m.txt"

499999500000.00


In [82]:
!nvprof ./Q1 "input_1m.txt"

==6897== NVPROF is profiling process 6897, command: ./Q1 input_1m.txt
499999500000.00
==6897== Profiling application: ./Q1 input_1m.txt
==6897== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.36%  1.6634ms         1  1.6634ms  1.6634ms  1.6634ms  [CUDA memcpy HtoD]
                   49.58%  1.6375ms         1  1.6375ms  1.6375ms  1.6375ms  ArrAdd(double*, int, double*)
                    0.06%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
      API calls:   96.39%  122.61ms         2  61.306ms  149.16us  122.46ms  cudaMalloc
                    2.78%  3.5398ms         2  1.7699ms  1.6603ms  1.8796ms  cudaMemcpy
                    0.43%  550.88us         2  275.44us  150.47us  400.42us  cudaFree
                    0.21%  269.49us         1  269.49us  269.49us  269.49us  cudaLaunchKernel
                    0.16%  198.74us       114  1.7430us     204ns  75.401us  cuDeviceGetAttribute
  

**Question2 (Q2)**

In [2]:

%%writefile Q2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>

using namespace std;


// Kernel definition
__global__ void BinarySearch(float *array, int size, float target, float *index, bool* found)
{

    *index = -1;
    int target_thread_id = -1;
    int search_part = (size-1) / blockDim.x + 1;

    if(*found == false) {

        int thread_id = threadIdx.x;
        extern __shared__ float shared_array[];


        float min_value = array[thread_id*search_part];
        float max_value = array[min( (thread_id+1) * search_part - 1, size)];

        if(target >= min_value && target <= max_value){

          *found = true;
          target_thread_id= thread_id;

          # if __CUDA_ARCH__>=200
             printf("%f %f %d \n", min_value, max_value, thread_id);
          #endif

          for(int i = 0; i < search_part; i++) {
            int shared_location = thread_id * search_part + i;
            if(shared_location < size) {
              shared_array[i] = array[shared_location];
            }
            else {
              shared_array[i] = INT_MAX;
            }
        }
      }

    __syncthreads();


     if (target_thread_id == thread_id){

      # if __CUDA_ARCH__>=200
             printf("%d \n", target_thread_id);
      #endif


        bool found_target = false;
        int start = 0;
        int end = search_part - 1;
        int mid = (start + end) / 2;

        while(start <= end && found_target == false)
        {
            if(shared_array[mid] < target) {
                start = mid + 1;
            }
            else if(shared_array[mid] > target) {
                end = mid - 1;
            }
            else {
               found_target = true;
               *index = mid + target_thread_id * search_part;


            }
            mid = (start + end) / 2;
        }
      }

    }
}


int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];
    float target = atof(argv[2]);

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

    float* array = nullptr;  // Pointer to the array
    float * result = nullptr;
    float value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        float* temp = static_cast<float*>(realloc(array, (count + 1) * sizeof(float)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();


      bool * flag;
     // host to device allocation
     float *array_device;
     float *result_device;
     bool * flag_device;

     result = (float*)malloc(sizeof(float));
     flag = (bool*)malloc(sizeof(bool));
     *flag = false;

     cudaMalloc((void**)&array_device, count * sizeof(float));
     cudaMalloc((void**)&result_device,sizeof(float));
     cudaMalloc((void**)&flag_device,sizeof(bool));

    cudaMemcpy(array_device, array, count * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(flag_device, flag, sizeof(bool), cudaMemcpyHostToDevice);



    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1,1);

    cout<<(threadsPerBlock.x)<<endl;
    cout<<(count)<<endl;
    cout<<(count-1) / threadsPerBlock.x + 1 <<endl;

    BinarySearch<<< numBlocks, threadsPerBlock , ((count-1) / threadsPerBlock.x +1 ) * sizeof(float) >>>(array_device,count, target,result_device, flag_device);

    cudaMemcpy( result, result_device, sizeof(float), cudaMemcpyDeviceToHost);

    cout<<"target index : "<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Writing Q2_2_31_2_33.cu


# updated without prints

In [50]:

%%writefile Q2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>

using namespace std;


// Kernel definition
__global__ void BinarySearch(float *array, int size, float target, float *index, bool* found)
{

    *index = -1;
    int target_thread_id = -1;
    int search_part = (size-1) / blockDim.x + 1;

    if(*found == false) {

        int thread_id = threadIdx.x;
        extern __shared__ float shared_array[];


        float min_value = array[thread_id*search_part];
        float max_value = array[min( (thread_id+1) * search_part - 1, size)];

        if(target >= min_value && target <= max_value){

          *found = true;
          target_thread_id= thread_id;


          for(int i = 0; i < search_part; i++) {
            int shared_location = thread_id * search_part + i;
            if(shared_location < size) {
              shared_array[i] = array[shared_location];
            }
            else {
              shared_array[i] = INT_MAX;
            }
        }
      }

    __syncthreads();


     if (target_thread_id == thread_id){

        bool found_target = false;
        int start = 0;
        int end = search_part - 1;
        int mid = (start + end) / 2;

        while(start <= end && found_target == false)
        {
            if(shared_array[mid] < target) {
                start = mid + 1;
            }
            else if(shared_array[mid] > target) {
                end = mid - 1;
            }
            else {
               found_target = true;
               *index = mid + target_thread_id * search_part;


            }
            mid = (start + end) / 2;
        }
      }

    }
}


int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];
    float target = atof(argv[2]);

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

    float* array = nullptr;  // Pointer to the array
    float * result = nullptr;
    float value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        float* temp = static_cast<float*>(realloc(array, (count + 1) * sizeof(float)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();


      bool * flag;
     // host to device allocation
     float *array_device;
     float *result_device;
     bool * flag_device;

     result = (float*)malloc(sizeof(float));
     flag = (bool*)malloc(sizeof(bool));
     *flag = false;

     cudaMalloc((void**)&array_device, count * sizeof(float));
     cudaMalloc((void**)&result_device,sizeof(float));
     cudaMalloc((void**)&flag_device,sizeof(bool));

    cudaMemcpy(array_device, array, count * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(flag_device, flag, sizeof(bool), cudaMemcpyHostToDevice);



    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1,1);

    BinarySearch<<< numBlocks, threadsPerBlock , ((count-1) / threadsPerBlock.x +1 ) * sizeof(float) >>>(array_device,count, target,result_device, flag_device);

    cudaMemcpy( result, result_device, sizeof(float), cudaMemcpyDeviceToHost);

    cout<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);
    free(flag);

    // Free device memory
    cudaFree(array_device);
    cudaFree(flag_device);
    cudaFree(result_device);

   return 0;
}


Overwriting Q2_2_31_2_33.cu


In [51]:
!chmod +x Q2_2_31_2_33.cu
!nvcc Q2_2_31_2_33.cu -o Q2

In [52]:
!./Q2 "input_10.txt" 9.0

9


In [53]:
!./Q2 "input_10.txt" 10.0

-1


In [54]:
!./Q2 "input_10.txt" 9.6

-1


In [55]:
!./Q2 "input_100.txt" 99.0

99


In [56]:
!./Q2 "input_100.txt" 56.0

56


In [57]:
!./Q2 "input_100.txt" 100.0

-1


In [58]:
!./Q2 "input_1000.txt" 300.0

300


In [59]:
!./Q2 "input_1000.txt" 999.0

999


In [60]:
!./Q2 "input_10th.txt" 5600.0

5600


In [61]:
!./Q2 "input_10th.txt" 9999.0

9999


In [62]:
!./Q2 "input_10th.txt" 10000.0

-1


In [63]:
!./Q2 "input_100th.txt" 98765.0

98765


In [64]:
!./Q2 "input_100th.txt" 100001.0

-1


In [65]:
!./Q2 "input_100th.txt" 588.0

588


In [66]:
!./Q2 "input_1m.txt" 900000.0

900000


In [67]:
!./Q2 "input_1m.txt" 900793.0

900793


In [68]:
!./Q2 "input_1m.txt" 1000002.0

-1


In [69]:
!./Q2 "input_1m.txt" 1000002.0

-1


In [70]:
!nvprof ./Q2 "input_1m.txt" 1000002.0

==4748== NVPROF is profiling process 4748, command: ./Q2 input_1m.txt 1000002.0
-1
==4748== Profiling application: ./Q2 input_1m.txt 1000002.0
==4748== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   98.72%  679.92us         2  339.96us     672ns  679.25us  [CUDA memcpy HtoD]
                    0.98%  6.7190us         1  6.7190us  6.7190us  6.7190us  BinarySearch(float*, int, float, float*, bool*)
                    0.31%  2.1120us         1  2.1120us  2.1120us  2.1120us  [CUDA memcpy DtoH]
      API calls:   98.32%  95.772ms         3  31.924ms  3.7470us  95.679ms  cudaMalloc
                    0.97%  940.91us         3  313.64us  22.070us  844.56us  cudaMemcpy
                    0.31%  305.94us         3  101.98us  6.0810us  199.75us  cudaFree
                    0.23%  228.75us         1  228.75us  228.75us  228.75us  cudaLaunchKernel
                    0.14%  135.62us       114  1.1890us     144ns  52.839us